In [1]:
import numpy as np;
import tensorflow as tf
from tensorflow.keras.datasets import imdb 
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense


In [2]:
max_features = 10000
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=max_features)


print(f'training the shape data: {X_train.shape}, label {y_train.shape}')
print(f'training the shape data: {X_test.shape}, label {y_test.shape}')



training the shape data: (25000,), label (25000,)
training the shape data: (25000,), label (25000,)


In [3]:

print(X_train[0])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [4]:
word_index = imdb.get_word_index()
reverse_str = {value:key for key , value in word_index.items()}
decoded_review = ' '.join(reverse_str.get(i-3, '?') for i in X_train[0])
decoded_review


"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [22]:
from tensorflow.keras.preprocessing import sequence
max_len = 500
X_train = sequence.pad_sequences(X_train,maxlen = max_len)
X_test = sequence.pad_sequences(X_test,maxlen = max_len)
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

In [23]:
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=100, input_shape=(max_len,)))
model.add(SimpleRNN(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


# model = Sequential()
# model.add(Embedding(input_dim=max_features, output_dim=100, input_shape=(max_len,)))
# model.add(SimpleRNN(100, activation='tanh'))
# model.add(Dense(1, activation='sigmoid'))


In [24]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 500, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,020,201 (3.89 MB)

 Trainable params: 1,020,201 (3.89 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
from tensorflow.keras.callbacks import EarlyStopping
earlyStopping = EarlyStopping(monitor='vol_loss',patience=5,restore_best_weights=True)
earlyStopping

In [27]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True)

model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 91s 145ms/step - accuracy: 0.8921 - loss: 0.2642 - val_accuracy: 0.8044 - val_loss: 0.4661
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 80s 128ms/step - accuracy: 0.9238 - loss: 0.1933 - val_accuracy: 0.7830 - val_loss: 0.4998
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 87s 140ms/step - accuracy: 0.9453 - loss: 0.1424 - val_accuracy: 0.7916 - val_loss: 0.6076
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 124ms/step - accuracy: 0.9624 - loss: 0.0992 - val_accuracy: 0.7816 - val_loss: 0.6689


In [ ]:
model.save('simple_run_imdb.h5')